# Preface
The goal is to create a recommendation system for MTG Players to be able to use. 

response = requests.get('https://api.scryfall.com/bulk-data') to get all data API endpoints 

import json
def jprint(obj):
    text = json.dumps(obj, sort_keys = True, indent = 4)
    print(text)
    
jprint(response.json())


* Brought in both oracle and card datasets. upon inspection, the oracle will be best to go with. Less repeated cards, easier to work with, unique id_identifiers

Ratio of missing values = the number of missing values / total number of observations * 100


mtgo_foil_id is not needed.
Flavor text is not needed. We can add a line about where to find all flavor text, or import into database at later time for search recommendations
security_stamp is not needed.
preview is not needed for modeling purposes
arena_id could be useful for an online rec system
watermark not needed
produced mana can be removed. If not, we need to replace NaN values with just Not Applicable
all parts could be viewed as a target for combo model
object not needed. every object is a card-type object
lang contains only 10 japanese cards out of 26000. Removed column.

type can help fix color column. We can use type to tell if it's an artifact, then create a colorless condition

mana_cost: could be fixed. It has a few cards that are duplicates. Dropped those. Then there are leftovers that are dual faced cards. Costs are different for each. We could fill these with cmc costs instead or look at cards individually for this issue. For now, dropped column.
* Fixed power, toughness, and edhrec_rank

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import requests
import pandas as pd

In [9]:
class Data_Handling(object):
    def wrangle(filepath):
        """
        Wrangles in filepath. Tells the difference between JSON and CSV and creates dataframe.
    
        NA Values are handled as:
    
        edhrec_rank: replaces na values with incremental counter from the last recommended value to the length of the dataframe.
        power and toughness: na values represent cards that are non creature. Replaced with a zero.
        """
        if filepath.endswith(".json"):
            df = pd.read_json(filepath)
        else:
            df = pd.read_csv(filepath)
        
        #Fix NA Values for edhrec_rank
        if 'edhrec_rank' in df.columns:
            edh_fix = df[df['edhrec_rank'].isna() == True]
            counter = 22665 # Max rank + 1
    
            edh_fix.edhrec_rank = range(counter, (counter + len(edh_fix)))
            df.loc[edh_fix.index, :] = edh_fix[:]
            df['edhrec_rank'] = df['edhrec_rank'].astype(int)
    
        # Fix power column
        if 'power' in df.columns:
            df['power'].loc[df['power'].isna() == True] = 0
        
        # Fix Toughness Columns
        if 'toughness' in df.columns:
            df['toughness'].loc[df['toughness'].isna() == True] = 0
            
        # Fix CMC Column
        if 'cmc' in df.columns:
            df['cmc'].loc[17411] = 1.0
            df['cmc'] = df['cmc'].astype(int)
        
        if 'oracle_id' in df.columns:
            df.set_index('oracle_id', inplace=True)
        
        if 'colors' in df.columns:
            df['colors'].fillna(0, inplace=True)
            # df['colors'] = df['colors'].str[0]
        # Clean up mana_cost column
        # if 'mana_cost' in df.columns:
        #     mana = df[df['mana_cost'].isna() == True]
        #     mana_drop = mana[mana['mtgo_id'].isna() == True]
        #     df.drop(labels= mana_drop.index, inplace=True)
        

        return df
    def drop_cols(df):
        # Drops all columns with greater than 35% NA values
        #Drops mtgo_id column, which has a high number of NA values as well
        drop_cols = [col for col in df.columns if (df[col].isna().sum() / len(df) *100) > 35]
        drop_cols.append('mtgo_id')
        df.drop(columns = drop_cols, inplace=True)
        
        return df
    
    def modeling_prep_mtg_oracle(df):
        # Drop columns for modeling purposes
        drop_cols = ['id', 'multiverse_ids', 'tcgplayer_id', 'cardmarket_id', 'lang', 'object', 
                     'released_at', 'uri', 'scryfall_uri', 'layout', 'highres_image', 'image_status', 
                     'image_uris', 'games', 'frame', 'full_art', 'textless', 'booster', 'story_spotlight', 'prices',
                     'legalities', 'reserved', 'foil', 'nonfoil', 'card_back_id', 'artist', 'artist_ids', 'illustration_id', 
                     'border_color', 'oversized', 'finishes', 'scryfall_set_uri', 'rulings_uri', 'promo', 'set_uri', 'set_search_uri', 
                     'reprint', 'variation', 'set_id', 'prints_search_uri', 'collector_number', 'digital']
        df.drop(columns = drop_cols, inplace= True)

        return df

In [10]:
df = Data_Handling.wrangle('https://c2.scryfall.com/file/scryfall-bulk/oracle-cards/oracle-cards-20220412211242.json')
df = Data_Handling.modeling_prep_mtg_oracle(df)
df = Data_Handling.drop_cols(df)
df.head()

,name,mana_cost,cmc,type_line,oracle_text,colors,color_identity,keywords,set,set_name,set_type,rarity,edhrec_rank,related_uris,power,toughness
oracle_id,,,,,,,,,,,,,,,,
0004ebd0-dfd6-4276-b4a6-de0003e94237,Static Orb,{3},3,Artifact,"As long as Static Orb is untapped, players can...",[],[],[],7ed,Seventh Edition,core,rare,2631,{'gatherer': 'https://gatherer.wizards.com/Pag...,0,0
0006faf6-7a61-426c-9034-579f2cfcfa83,Sensory Deprivation,{U},1,Enchantment — Aura,Enchant creature\nEnchanted creature gets -3/-0.,[U],[U],[Enchant],m14,Magic 2014,core,common,21574,{'gatherer': 'https://gatherer.wizards.com/Pag...,0,0
0007c283-5b7a-4c00-9ca1-b455c8dff8c3,Road of Return,{G}{G},2,Sorcery,Choose one —\n• Return target permanent card f...,[G],[G],[Entwine],c19,Commander 2019,commander,rare,4080,{'gatherer': 'https://gatherer.wizards.com/Pag...,0,0
000d5588-5a4c-434e-988d-396632ade42c,Storm Crow,{1}{U},2,Creature — Bird,Flying (This creature can't be blocked except ...,[U],[U],[Flying],9ed,Ninth Edition,core,common,12416,{'gatherer': 'https://gatherer.wizards.com/Pag...,1,2
000e5d65-96c3-498b-bd01-72b1a1991850,Walking Sponge,{1}{U},2,Creature — Sponge,{T}: Target creature loses your choice of flyi...,[U],[U],[],ulg,Urza's Legacy,expansion,uncommon,18864,{'gatherer': 'https://gatherer.wizards.com/Pag...,1,1


In [120]:
df['colors'] = df['colors'].str[0]
df['colors'] = df['colors'].str.replace('U', 'B')

In [94]:
s = df['colors'].str.replace('[','').replace(']', '')
print(s)

oracle_id
0004ebd0-dfd6-4276-b4a6-de0003e94237   NaN
0006faf6-7a61-426c-9034-579f2cfcfa83   NaN
0007c283-5b7a-4c00-9ca1-b455c8dff8c3   NaN
000d5588-5a4c-434e-988d-396632ade42c   NaN
000e5d65-96c3-498b-bd01-72b1a1991850   NaN
                                        ..
fffa9334-3576-4f70-9605-2ad062cdbc69   NaN
fffaa634-42a0-4038-b027-24f28754fec7   NaN
fffcb71f-e802-436e-917d-eaa6607ab74f   NaN
fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91   NaN
ffff90c3-63c4-4dee-a21d-6b2b113f4f80   NaN
Name: colors, Length: 25292, dtype: float64


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25292 entries, 0004ebd0-dfd6-4276-b4a6-de0003e94237 to ffff90c3-63c4-4dee-a21d-6b2b113f4f80
Data columns (total 36 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               25292 non-null  object
 1   mana_cost          24307 non-null  object
 2   cmc                25292 non-null  int32 
 3   type_line          25292 non-null  object
 4   oracle_text        24162 non-null  object
 5   colors             24307 non-null  object
 6   color_identity     25292 non-null  object
 7   keywords           25292 non-null  object
 8   foil               25292 non-null  bool  
 9   nonfoil            25292 non-null  bool  
 10  finishes           25292 non-null  object
 11  oversized          25292 non-null  bool  
 12  promo              25292 non-null  bool  
 13  reprint            25292 non-null  bool  
 14  variation          25292 non-null  bool  
 15  set_id             25292 n

In [19]:
mana = df[df['mana_cost'].isna() == True]
mana_keep = mana[mana['mtgo_id'].isna() == False]
mana_drop = mana[mana['mtgo_id'].isna() == True]

mana_keep[mana_keep['mana_cost'].isna() == True]

,object,id,oracle_id,multiverse_ids,mtgo_id,mtgo_foil_id,tcgplayer_id,cardmarket_id,name,lang,...,all_parts,card_faces,frame_effects,tcgplayer_etched_id,promo_types,loyalty,life_modifier,hand_modifier,color_indicator,content_warning
257,card,0dbaef61-fa39-4ea7-bc21-445401c373e7,0272ca81-e727-4f4b-b06e-072d70bb5558,"[414479, 414480]",61462.0,61463.0,120122.0,290979.0,Ulvenwald Captive // Ulvenwald Abomination,en,...,NaN,"[{'object': 'card_face', 'name': 'Ulvenwald Ca...",[mooneldrazidfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
519,card,c1f53d7a-9dad-46e8-b686-cd1362867445,04eeb9ad-5c59-411b-8809-db8349838588,"[410049, 410050]",59810.0,59811.0,115917.0,288984.0,"Westvale Abbey // Ormendahl, Profane Prince",en,...,"[{'object': 'related_card', 'id': 'c1f53d7a-9d...","[{'object': 'card_face', 'name': 'Westvale Abb...",[sunmoondfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
739,card,d2704743-2e23-40b9-a367-c73d2db45afc,0761a0e7-d443-4bab-bb15-307c83d4a6a1,"[534783, 534784]",92980.0,NaN,248261.0,575012.0,Lunarch Veteran // Luminous Phantom,en,...,NaN,"[{'object': 'card_face', 'name': 'Lunarch Vete...",[sunmoondfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
852,card,b6867ddd-f953-41c6-ba36-86ae2c14c908,08b3328c-1d96-4a05-ae8b-f1b654084faa,"[414313, 414314]",61316.0,61317.0,120485.0,291198.0,Extricator of Sin // Extricator of Flesh,en,...,"[{'object': 'related_card', 'id': 'b6867ddd-f9...","[{'object': 'card_face', 'name': 'Extricator o...",[mooneldrazidfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
991,card,965e6bd5-dc32-406c-bc99-ceb15be4d3f2,0a154fb2-9f23-4c22-baee-728492385d6d,"[535011, 535012]",93436.0,NaN,247911.0,574768.0,Devoted Grafkeeper // Departed Soulkeeper,en,...,NaN,"[{'object': 'card_face', 'name': 'Devoted Graf...",[sunmoondfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24918,card,031c5cff-e579-432a-bcee-864b12eb0558,fc58a086-2743-4465-bf16-4a20701cd3b1,"[540860, 540861]",94348.0,NaN,253720.0,583521.0,Panicked Bystander // Cackling Culprit,en,...,NaN,"[{'object': 'card_face', 'name': 'Panicked Bys...",[sunmoondfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
25192,card,228e551e-023a-4c9a-8f32-58dae6ffdf7f,ff0ab867-b710-4b1a-baed-95fc3cf68f79,[491818],83329.0,NaN,221774.0,494569.0,Valakut Awakening // Valakut Stoneforge,en,...,NaN,"[{'object': 'card_face', 'name': 'Valakut Awak...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25265,card,d4054ae6-0227-4d99-8cb5-72e8b5d0b726,ffc1fe11-0ecb-4497-a266-0817c79477e9,"[534901, 534902]",93216.0,NaN,248224.0,575042.0,Vengeful Strangler // Strangling Grasp,en,...,NaN,"[{'object': 'card_face', 'name': 'Vengeful Str...",[sunmoondfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
25274,card,0d200f98-3377-46a3-9197-3cbd95d03dbf,ffd3c928-af42-4d69-8d22-a4c72f93b1c7,"[410021, 410022]",60392.0,60393.0,116492.0,289129.0,Harvest Hand // Scrounged Scythe,en,...,NaN,"[{'object': 'card_face', 'name': 'Harvest Hand...",[sunmoondfc],NaN,NaN,NaN,NaN,NaN,NaN,NaN
